In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [43]:
train = pd.read_csv('train.csv').drop(['S.No', 'time'], axis=1)
test = pd.read_csv('test.csv').drop(['S.No'], axis=1)

In [44]:
## reference: used from LAB exercise 
def confusion_matrix(true_labels, pred_labels):
    """ To calculate confusion matrix of model built"""
    matrix = np.zeros((3, 3))
    for (true, pred) in zip(true_labels, pred_labels):
        matrix[int(true - 1), int(pred - 1)] += 1
    return matrix

In [45]:
def split_dataset(df):
    idx = int(df.shape[0]*70/100)
    ##normalization
    df.iloc[:,:-1]=(df.iloc[:,:-1]-df.iloc[:,:-1].min())/(df.iloc[:,:-1].max()-df.iloc[:,:-1].min())
    return df.iloc[:idx,:-1], df.iloc[:idx,-1].values, df.iloc[idx:,:-1], df.iloc[idx:,-1].values
def accuracy_score(Predict, y_val):
    return (sum(a == b for a,b in zip(Predict.tolist(), y_val.tolist()))/len(y_val))
train_x, train_y, test_x, test_y = split_dataset(train)


In [46]:
X_train, y_train, X_test, y_test = split_dataset(train)

In [47]:
y_t = y_train.copy()
y_train = pd.get_dummies(y_train).values

In [48]:
def Loss_function(X, Y, w): # Cross-entropy loss
    cost = 0 
    n = X.shape[0]
    for i in range(n):  
        cost -= 1/n * np.matmul(Y[i],np.log(softmax(np.matmul(w, X[i])) )) 
    return cost

In [49]:
def softmax(x): 
    return np.exp(x) / np.sum(np.exp(x)) ## To calculate probability

In [50]:
def Logistic_Regression(X,Y,learning_rate, iteration=10000, lambda_ = 0.01):
    n = X.shape[0] #number of samples
    X = np.insert(X,0,1,axis=1) # Add bias term as constant value in training set
    
    weight = np.zeros((Y.shape[1], X.shape[1])) 
    loss = [] 
    
    for k in range(iteration):

        Loss = Loss_function(X, Y, weight) 
        print('Loss', Loss) 
        loss.append(Loss) 

        iterate = np.random.permutation(n) 
        for i in iterate:    
            P = softmax(np.matmul(weight, X[i]) + 2 * lambda_ * np.sum(np.absolute(weight))) # L2 regularizer
            gradient = np.outer(P - Y[i], X[i]) # sgd
            weight = weight - learning_rate * gradient 
           
    return weight, loss 



In [51]:
def predict(X_test, weight):
    X_test = np.insert(np.array(X_test), 0, 1, axis=1) # adding bias
    predictions = [] # Define a prediction list
    for x in range(X_test.shape[0]):
        prob = np.argmax(softmax(np.matmul(weight, X_test[x]))) 
        predictions.append(prob) 
    return predictions

In [ ]:
w, loss = Logistic_Regression(np.array(X_train), np.array(y_train), 0.002)

In [ ]:
pd.Series(loss).plot(xlabel='iteration', ylabel = 'Loss')

In [55]:
ypred_test = predict(X_test, w)
accuracy_score(pd.Series(ypred_test), y_test)

0.8026940256839754

In [57]:
ypred_train = predict(X_train, w)
accuracy_score(pd.Series(ypred_train), y_t)

0.8064728403924384

In [59]:
test = pd.read_csv('test.csv')#.drop(['S.No'], axis=1)
submission = pd.DataFrame()
submission['S.No'] = test['S.No']
test = test.drop(['time','S.No'], axis=1)
test=np.array((test-test.min())/(test.max()-test.min()))
submission['LABELS'] = predict(test, w)
#submission.to_csv('logistic_submission/submission.csv', index=False)